<a href="https://colab.research.google.com/github/alan-medina-gomez-lic/Evidencia1_Ago2025/blob/main/evidencia_2_Estructura_de_datos_y_su_procesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install my_library

 instalacion de librerias

In [32]:
import pandas as pd
from datetime import datetime, timedelta
import os
import json
import openpyxl
from openpyxl.styles import Font, Border, Side, Alignment
import matplotlib.pyplot as plt
import numpy as np

# --- CONSTANTES GLOBALES ---
TURNOS_POSIBLES = ['MATUTINO', 'VESPERTINO', 'NOCTURNO']
ARCHIVO_ESTADO = "estado_reservas_pandas.json"

class SistemaReservasPandas:
    def __init__(self):
        self.contador_clientes = 1
        self.contador_salas = 1
        self.contador_folios = 1

        # Inicializar DataFrames vacíos con las columnas esperadas
        self.CLIENTES_DF = pd.DataFrame(columns=['Nombre', 'Apellidos'])
        self.SALAS_DF = pd.DataFrame(columns=['Nombre', 'Cupo'])
        self.RESERVACIONES_DF = pd.DataFrame(columns=['Cliente_ID', 'Sala_ID', 'Fecha', 'Turno', 'Evento'])

        self.cargar_estado()

    # --- 1. PERSISTENCIA Y ESTADO ---

    def __init_empty_dfs(self):
        """Inicializa los DataFrames y contadores a estado vacío."""
        self.contador_clientes = 1
        self.contador_salas = 1
        self.contador_folios = 1
        self.CLIENTES_DF = pd.DataFrame(columns=['Nombre', 'Apellidos'])
        self.SALAS_DF = pd.DataFrame(columns=['Nombre', 'Cupo'])
        self.RESERVACIONES_DF = pd.DataFrame(columns=['Cliente_ID', 'Sala_ID', 'Fecha', 'Turno', 'Evento'])

    def guardar_estado(self):
        """Almacena el estado de la solución en JSON (memoria no volátil)."""
        estado = {
            'contador_clientes': self.contador_clientes,
            'contador_salas': self.contador_salas,
            'contador_folios': self.contador_folios,
            'clientes': self.CLIENTES_DF.to_json(orient='split'),
            'salas': self.SALAS_DF.to_json(orient='split'),
            'reservaciones': self.RESERVACIONES_DF.to_json(orient='split')
        }

        try:
            with open(ARCHIVO_ESTADO, 'w') as f:
                json.dump(estado, f, indent=4)
            print("Estado de la solución almacenado exitosamente.")
        except Exception as e:
            print(f"Error al guardar el estado: {e}")

    def cargar_estado(self):
        """Recupera el estado anterior de la solución si existe."""
        if os.path.exists(ARCHIVO_ESTADO):
            try:
                with open(ARCHIVO_ESTADO, 'r') as f:
                    estado = json.load(f)

                self.contador_clientes = estado.get('contador_clientes', 1)
                self.contador_salas = estado.get('contador_salas', 1)
                self.contador_folios = estado.get('contador_folios', 1)

                self.CLIENTES_DF = pd.read_json(estado['clientes'], orient='split')
                self.SALAS_DF = pd.read_json(estado['salas'], orient='split')
                self.RESERVACIONES_DF = pd.read_json(estado['reservaciones'], orient='split')

                self.SALAS_DF.index = self.SALAS_DF.index.astype(str)
                if 'Cupo' in self.SALAS_DF.columns:
                     self.SALAS_DF['Cupo'] = self.SALAS_DF['Cupo'].astype(int)
                if 'Sala_ID' in self.RESERVACIONES_DF.columns:
                     self.RESERVACIONES_DF['Sala_ID'] = self.RESERVACIONES_DF['Sala_ID'].astype(str)

                print("Estado anterior de la solución recuperado con éxito.")
            except Exception as e:
                print(f"Error al cargar el estado: {e}. Se inicia con un estado inicial vacío.")
                self.__init_empty_dfs()
        else:
            print("No se encontró estado anterior. Se inicia con un estado inicial vacío.")

    # --- 2. REGISTROS DE ENTIDADES ---

    def registrar_sala(self, nombre, cupo):
        """Registra una nueva sala."""
        try:
            # Validación de tipo (asumiendo que 'cupo' ya fue validado como isdigit en el menú)
            cupo_int = int(cupo)
            if cupo_int <= 0:
                print("Error: El cupo debe ser un número entero positivo.")
                return False
        except ValueError:
            print("Error: El cupo debe ser un número válido.")
            return False

        nueva_clave = str(self.contador_salas)
        nueva_sala_data = pd.DataFrame({'Nombre': [nombre.strip().upper()], 'Cupo': [cupo_int]}, index=[nueva_clave])

        self.SALAS_DF = pd.concat([self.SALAS_DF, nueva_sala_data])
        self.contador_salas += 1

        print(f"Sala '{nombre.strip().upper()}' registrada con éxito. Clave: {nueva_clave}, Cupo: {cupo_int}.")
        return True

    def registrar_cliente(self, nombre, apellidos):
        """Registra un nuevo cliente."""

        nueva_clave = f'C{self.contador_clientes}'

        nombre_limpio = nombre.strip()
        apellidos_limpios = apellidos.strip()

        nuevo_cliente_data = pd.DataFrame({'Nombre': [nombre_limpio], 'Apellidos': [apellidos_limpios]}, index=[nueva_clave])

        self.CLIENTES_DF = pd.concat([self.CLIENTES_DF, nuevo_cliente_data])
        self.contador_clientes += 1

        print(f"Cliente '{nombre_limpio} {apellidos_limpios}' registrado con éxito. Clave: {nueva_clave}.")
        return True

    # --- 3. RESERVACIONES ---

    def validar_fecha(self, fecha_str):
        """Valida el formato de fecha (DD/MM/AAAA)."""
        try:
            fecha_reserva = datetime.strptime(fecha_str, '%d/%m/%Y').date()
        except ValueError:
            print("Error: Formato de fecha incorrecto. Use el formato DD/MM/AAAA (ej. 20/10/2025).")
            return None
        return fecha_reserva

    def seleccionar_cliente(self):
        """Muestra clientes ordenados y solicita selección."""
        if self.CLIENTES_DF.empty:
            print("No hay clientes registrados. Debe registrar un cliente primero.")
            return None

        while True:
            clientes_ordenados = self.CLIENTES_DF.sort_values(by=['Apellidos', 'Nombre'])

            print("\n--- CLIENTES REGISTRADOS (Clave: Apellidos, Nombre) ---")
            for index, row in clientes_ordenados.iterrows():
                print(f"[{index}]: {row['Apellidos']}, {row['Nombre']}")
            print("-----------------------------------------------------")

            clave_ingresada = input("Ingrese la CLAVE del cliente (ej. C1) o 'C' para cancelar: ").strip().upper()

            if clave_ingresada == 'C':
                print("Operación de reserva cancelada.")
                return None

            if not clave_ingresada: # Validación de no vacío
                print("Error: La clave del cliente no puede estar vacía. Intente de nuevo.")
                continue

            if clave_ingresada in self.CLIENTES_DF.index:
                return clave_ingresada
            else:
                print(f"La clave '{clave_ingresada}' no existe. Intente de nuevo.")

    def registrar_reservacion(self):
        """Registra una reservación con todas las validaciones."""
        print("\n\n*** REGISTRAR NUEVA RESERVACIÓN ***")

        if self.CLIENTES_DF.empty or self.SALAS_DF.empty:
            print("Error: No hay clientes o salas registradas.")
            return

        cliente_id = self.seleccionar_cliente()
        if cliente_id is None:
            return

        # 1. Pedir y validar FECHA
        while True:
            fecha_str = input("Ingrese la FECHA de la reserva (DD/MM/AAAA): ").strip()

            if not fecha_str:
                print("Error: La fecha no puede estar vacía o contener solo espacios. Intente de nuevo.")
                continue

            fecha_reserva = self.validar_fecha(fecha_str)

            if fecha_reserva:
                fecha_actual_sistema = datetime.now().date()
                fecha_minima_reserva = fecha_actual_sistema + timedelta(days=2)

                if fecha_reserva >= fecha_minima_reserva:
                    fecha_almacenar = fecha_reserva.strftime('%Y-%m-%d')
                    break
                else:
                    print(f"La fecha por reservar debe ser, por lo menos, dos días posteriores a la fecha actual del sistema. (Fecha mínima: {fecha_minima_reserva.strftime('%d/%m/%Y')}).")

        # 2. Consultar disponibilidad y elegir Sala/Turno
        query_str = f"Fecha == '{fecha_almacenar}'"
        reservas_del_dia = self.RESERVACIONES_DF.query(query_str, inplace=False).copy()

        if not reservas_del_dia.empty:
            reservas_del_dia['Sala_ID'] = reservas_del_dia['Sala_ID'].astype(str)
            reservas_del_dia['Turno'] = reservas_del_dia['Turno'].astype(str).str.upper()

        slots_ocupados = set(zip(reservas_del_dia['Sala_ID'], reservas_del_dia['Turno']))
        slots_disponibles = []

        print(f"\n--- DISPONIBILIDAD PARA EL DÍA {fecha_reserva.strftime('%d/%m/%Y')} ---")
        print("CLAVE | NOMBRE | CUPO | TURNO(S) DISPONIBLE(S)")
        print("-------------------------------------------------")

        for sala_id, sala_info in self.SALAS_DF.iterrows():
            turnos_disponibles_sala = []
            for turno in TURNOS_POSIBLES:
                slot = (sala_id, turno)
                if slot not in slots_ocupados:
                    turnos_disponibles_sala.append(turno)
                    slots_disponibles.append(slot)

            if turnos_disponibles_sala:
                print(f"{sala_id:<5} | {sala_info['Nombre']:<6} | {sala_info['Cupo']:<4} | {', '.join(turnos_disponibles_sala)}")
            else:
                print(f"{sala_id:<5} | {sala_info['Nombre']:<6} | {sala_info['Cupo']:<4} | (COMPLETO)")

        if not slots_disponibles:
            print("\nLo sentimos, no hay salas disponibles para esa fecha.")
            return

        # Lógica para determinar qué turnos aún puede reservar el cliente
        turnos_reservados_por_el_cliente_hoy = reservas_del_dia.query(f"Cliente_ID == '{cliente_id}'")['Turno'].unique().tolist()
        turnos_disponibles_para_cliente = [t for t in TURNOS_POSIBLES if t not in turnos_reservados_por_el_cliente_hoy]

        if not turnos_disponibles_para_cliente:
            print("\nError: Usted ya ha reservado una sala para todos los turnos posibles en esta fecha. No puede realizar más reservas.")
            return

        turnos_prompt_texto = ", ".join(turnos_disponibles_para_cliente)


        slot_elegido = None

        while slot_elegido is None:
            sala_elegida = None

            # Sub-loop para forzar la selección válida de Sala ID (numérica)
            while sala_elegida is None:
                entrada_sala = input("\nIngrese la CLAVE de la sala a reservar (o 'C' para cancelar): ").strip().upper()

                if entrada_sala == 'C':
                    print("Operación de reserva de sala/turno cancelada.")
                    return

                if not entrada_sala.strip() or not entrada_sala.isdigit():
                    print("Error: La clave de sala debe ser un número (ej. 1). Intente de nuevo.")
                    continue

                if entrada_sala.strip() not in self.SALAS_DF.index:
                    print(f"Error: La clave de sala '{entrada_sala.strip()}' no existe. Intente de nuevo.")
                    continue

                sala_elegida = entrada_sala.strip() # Sala ID válido y seleccionado

            # Sub-loop para forzar la selección válida del Turno para la Sala elegida
            while slot_elegido is None:
                turno_elegido = input(f"Ingrese el TURNO ({turnos_prompt_texto}): ").strip().upper()

                if not turno_elegido:
                    print("Error: El turno no puede estar vacío o contener solo espacios. Vuelva a intentar.")
                    continue

                # Validation 1: Client conflict/Valid Turno list
                if turno_elegido not in turnos_disponibles_para_cliente:
                    if turno_elegido in turnos_reservados_por_el_cliente_hoy:
                        # Este error solo fuerza a re-seleccionar el TURNO, no la Sala
                        print(f"Error: El cliente ya tiene una reserva para el turno '{turno_elegido}'. Seleccione uno de los turnos disponibles: {turnos_prompt_texto}.")
                    else:
                        print(f"Error: El turno '{turno_elegido}' no es válido. Los turnos posibles son: {turnos_prompt_texto}.")
                    continue # Vuelve a pedir el TURNO

                # Validation 2: General availability (Slot occupied by another client)
                slot_intento = (sala_elegida, turno_elegido)
                if slot_intento in slots_disponibles:
                    slot_elegido = slot_intento
                else:
                    print("Combinación de Sala/Turno no disponible o inválida. Revise el listado de disponibilidad, es posible que ya esté COMPLETO.")
                    break # Rompe el sub-loop del Turno y regresa a pedir la CLAVE de la sala de nuevo (flujo correcto para reintentar combinación)

        # FIN DE LOS LOOPS

        sala_id_final, turno_final = slot_elegido

        # 3. Pedir y validar Nombre del Evento
        while True:
            nombre_evento = input("Ingrese el NOMBRE del evento: ").strip().upper()
            if nombre_evento:
                break
            print("El nombre del evento no puede estar vacío.")

        # 4. Registrar
        folio_unico = f'F{self.contador_folios}'

        nueva_reserva_data = pd.DataFrame({'Cliente_ID': [cliente_id], 'Sala_ID': [sala_id_final], 'Fecha': [fecha_almacenar], 'Turno': [turno_final], 'Evento': [nombre_evento]}, index=[folio_unico])

        self.RESERVACIONES_DF = pd.concat([self.RESERVACIONES_DF, nueva_reserva_data])
        self.contador_folios += 1

        print(f"\nRESERVACIÓN REGISTRADA CON ÉXITO. FOLIO: {folio_unico}")

    def editar_reservacion(self):
        """Permite editar el nombre del evento buscando por rango de fechas."""
        if self.RESERVACIONES_DF.empty:
            print("No hay reservaciones registradas para editar.")
            return

        print("\n\n*** EDITAR NOMBRE DE EVENTO ***")

        # 1. Pedir y validar FECHAS del rango
        while True:
            fecha_inicio_str = input("Ingrese FECHA DE INICIO del rango (DD/MM/AAAA): ").strip()
            if not fecha_inicio_str:
                print("Error: La fecha de inicio no puede estar vacía. Intente de nuevo.")
                continue

            fecha_fin_str = input("Ingrese FECHA DE FIN del rango (DD/MM/AAAA): ").strip()
            if not fecha_fin_str:
                print("Error: La fecha de fin no puede estar vacía. Intente de nuevo.")
                continue

            try:
                fecha_inicio = datetime.strptime(fecha_inicio_str, '%d/%m/%Y').strftime('%Y-%m-%d')
                fecha_fin = datetime.strptime(fecha_fin_str, '%d/%m/%Y').strftime('%Y-%m-%d')
                if fecha_inicio <= fecha_fin:
                    break
                else:
                    print("La fecha de inicio debe ser anterior o igual a la fecha de fin.")
            except ValueError:
                print("Error: Formato de fecha incorrecto. Use DD/MM/AAAA.")

        query_str_rango = f"Fecha >= '{fecha_inicio}' and Fecha <= '{fecha_fin}'"
        eventos_en_rango = self.RESERVACIONES_DF.query(query_str_rango, inplace=False)

        if eventos_en_rango.empty:
            print(f"No se encontraron eventos entre {fecha_inicio_str} y {fecha_fin_str}.")
            return

        df_mostrar = eventos_en_rango[['Evento', 'Fecha']].copy()
        df_mostrar.index.name = 'FOLIO'

        # 2. Elegir Folio y editar Evento
        while True:
            print("\n--- EVENTOS ENCONTRADOS ---")
            print(df_mostrar)
            print("----------------------------")

            # CORRECCIÓN: Asegurar .upper() para el Folio
            folio_a_modificar = input("Ingrese el FOLIO del evento a modificar (ej. F1) o 'C' para cancelar: ").strip().upper()

            if folio_a_modificar == 'C':
                print("Edición cancelada.")
                return

            if not folio_a_modificar: # Validación de no vacío
                print("Error: El folio no puede estar vacío. Intente de nuevo.")
                continue

            if folio_a_modificar in eventos_en_rango.index:
                while True:
                    nuevo_evento = input(f"Ingrese el nuevo nombre para el evento '{folio_a_modificar}': ").strip().upper()
                    if nuevo_evento:
                        break
                    print("El nombre del evento no puede omitirse.")

                self.RESERVACIONES_DF.loc[folio_a_modificar, 'Evento'] = nuevo_evento
                print(f"Evento con FOLIO {folio_a_modificar} modificado a: '{nuevo_evento}'.")
                return
            else:
                print(f"Folio '{folio_a_modificar}' no encontrado en el rango de fechas. Intente de nuevo.")

    # --- 4. CONSULTA, REPORTE Y EXPORTACIÓN ---

    def consultar_reservaciones(self):
        """Genera y muestra el reporte tabular."""
        print("\n\n*** CONSULTAR RESERVACIONES ***")

        if self.RESERVACIONES_DF.empty:
            print("No hay reservaciones registradas para consultar.")
            return

        # 1. Pedir y validar FECHA de consulta
        while True:
            fecha_str = input("Ingrese la FECHA a consultar (DD/MM/AAAA): ").strip()

            if not fecha_str:
                print("Error: La fecha de consulta no puede estar vacía o contener solo espacios. Intente de nuevo.")
                continue

            try:
                fecha_consulta = datetime.strptime(fecha_str, '%d/%m/%Y').strftime('%Y-%m-%d')
                break
            except ValueError:
                print("Error: Formato de fecha incorrecto. Use DD/MM/AAAA.")


        query_str_consulta = f"Fecha == '{fecha_consulta}'"
        reservas_del_dia = self.RESERVACIONES_DF.query(query_str_consulta, inplace=False)

        if reservas_del_dia.empty:
            print(f"No hay reservaciones para la fecha {fecha_str}.")
            return

        reservas_del_dia = reservas_del_dia.copy()

        # Merge data from other DFs to build the report
        try:
            reservas_del_dia['CLIENTE'] = reservas_del_dia['Cliente_ID'].apply(
                lambda x: f"{self.CLIENTES_DF.loc[x]['Apellidos']}, {self.CLIENTES_DF.loc[x]['Nombre']}"
            )
        except KeyError:
             reservas_del_dia['CLIENTE'] = 'Cliente Desconocido (Error de ID)'

        reservas_del_dia['SALA'] = reservas_del_dia['Sala_ID']

        reporte_df = reservas_del_dia[['SALA', 'CLIENTE', 'Evento', 'Turno']].copy()

        # Generar reporte en consola
        fecha_reporte = datetime.strptime(fecha_consulta, '%Y-%m-%d').strftime('%d/%m/%Y')
        ANCHO_TOTAL = 100

        print("\n" + "*" * ANCHO_TOTAL)
        texto_encabezado = f"REPORTE DE RESERVACIONES PARA EL DÍA {fecha_reporte}"
        print(f"** {texto_encabezado:^94} **")
        print("*" * ANCHO_TOTAL)

        print(f"{'SALA':<5} {'CLIENTE':<30} {'EVENTO':<45} {'TURNO':<15}")
        print("*" * ANCHO_TOTAL)

        for _, row in reporte_df.iterrows():
            sala = str(row['SALA'])[:4]
            cliente = str(row['CLIENTE'])[:29]
            evento = str(row['Evento'])[:44]
            turno = str(row['Turno'])[:14]

            print(f"{sala:<5} {cliente:<30} {evento:<45} {turno:<15}")

        print("*" * 17 + " FIN DEL REPORTE " + "*" * (ANCHO_TOTAL - 17 - 17 - 3) + "\n")

        self.mostrar_imagen_reporte()
        self.submenu_exportacion(fecha_consulta, reporte_df)


    def submenu_exportacion(self, fecha_consulta_str, reporte_df):
        """Submenú para exportar el reporte."""
        while True:
            exportar = input("\n¿Desea exportar el reporte? (C=CSV, J=JSON, E=Excel, N=No): ").strip().upper()
            if exportar == 'C':
                self.exportar_a_csv(fecha_consulta_str, reporte_df)
                break
            elif exportar == 'J':
                self.exportar_a_json(fecha_consulta_str, reporte_df)
                break
            elif exportar == 'E':
                self.exportar_a_excel(fecha_consulta_str, reporte_df)
                break
            elif exportar == 'N':
                break
            else:
                print("Opción inválida. Intente de nuevo.")

    def exportar_a_csv(self, fecha_consulta_str, reporte_df):
        """Exporta el reporte a CSV."""
        archivo_nombre = f"Reporte_Reservas_{fecha_consulta_str}.csv"
        try:
            reporte_df.to_csv(archivo_nombre, index=False, encoding='utf-8')
            print(f"Reporte exportado exitosamente a CSV: {archivo_nombre}")
        except Exception as e:
            print(f"Error al guardar el archivo CSV: {e}")

    def exportar_a_json(self, fecha_consulta_str, reporte_df):
        """Exporta el reporte a JSON."""
        archivo_nombre = f"Reporte_Reservas_{fecha_consulta_str}.json"
        try:
            reporte_df.to_json(archivo_nombre, orient='records', indent=4)
            print(f"Reporte exportado exitosamente a JSON: {archivo_nombre}")
        except Exception as e:
            print(f"Error al guardar el archivo JSON: {e}")

    def exportar_a_excel(self, fecha_consulta_str, reporte_df):
        """Exporta el reporte a Excel con formato específico usando openpyxl."""
        archivo_nombre = f"Reporte_Reservas_{fecha_consulta_str}.xlsx"

        wb = openpyxl.Workbook()
        ws = wb.active
        headers = reporte_df.columns.tolist()

        font_bold = Font(bold=True)
        thin_border = Side(border_style="thin", color="000000")
        thick_border = Side(border_style="thick", color="000000")
        border_bottom_thick = Border(top=thin_border, left=thin_border, right=thin_border, bottom=thick_border)
        center_alignment = Alignment(horizontal='center', vertical='center')

        fecha_display = datetime.strptime(fecha_consulta_str, '%Y-%m-%d').strftime('%d/%m/%Y')

        # 1. Título (Negrita y Centrado, Fila 1)
        titulo = f"REPORTE DE RESERVACIONES PARA EL DÍA {fecha_display}"
        ws['A1'] = titulo
        ws['A1'].font = font_bold
        ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=len(headers))
        ws['A1'].alignment = center_alignment

        # 2. Encabezados (Negrita, Borde Inferior Grueso, Fila 2)
        ws.append(headers)
        header_row_num = ws.max_row

        for col_idx, header_text in enumerate(headers):
            col_letter = openpyxl.utils.get_column_letter(col_idx + 1)
            header_cell = ws[f'{col_letter}{header_row_num}']

            header_cell.font = font_bold
            header_cell.border = border_bottom_thick
            header_cell.alignment = center_alignment

            # Ajustar ancho de columna
            ws.column_dimensions[col_letter].width = max(len(header_text), 15) * 1.5

        # 3. Datos (Centrados)
        for row_data in reporte_df.values.tolist():
            ws.append(row_data)
            data_row_num = ws.max_row

            for cell in ws[data_row_num]:
                cell.alignment = center_alignment

        try:
            wb.save(archivo_nombre)
            print(f"Reporte exportado exitosamente a Excel: {archivo_nombre}")
        except Exception as e:
            print(f"Error al guardar el archivo Excel: {e}")

    # --- 5. VISUALIZACIÓN (Matplotlib) ---

    def generar_grafica_reservas_mensuales(self):
        """Genera y muestra un gráfico de barras de reservaciones por mes."""
        if self.RESERVACIONES_DF.empty:
            print("No hay reservaciones registradas para generar el gráfico.")
            return

        print("\nGenerando gráfico de reservas mensuales...")

        df = self.RESERVACIONES_DF.copy()

        df['fecha_dt'] = pd.to_datetime(df['Fecha'], format='%Y-%m-%d', errors='coerce')
        df.dropna(subset=['fecha_dt'], inplace=True)

        df['anio_mes'] = df['fecha_dt'].dt.to_period('M').astype(str)
        conteo_mensual = df['anio_mes'].value_counts().sort_index()

        plt.figure(figsize=(10, 6))
        ax = conteo_mensual.plot(kind='bar', color='#007BFF')

        plt.title('Número de Reservaciones por Mes', fontsize=16)
        plt.xlabel('Periodo (Año-Mes)', fontsize=12)
        plt.ylabel('Total de Reservaciones', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', linestyle='--', alpha=0.7)

        for container in ax.containers:
            ax.bar_label(container)

        plt.tight_layout()
        plt.show()

    # --- 6. AYUDA VISUAL ---

    def mostrar_imagen_reporte(self):
        """Muestra la referencia visual del reporte tabular."""
        print("\n--- IMAGEN DE REFERENCIA (REPORTE TABULAR) ---")
        print("---------------------------------------------")


# --- FUNCIONES DE SOPORTE Y MENÚ ---

def menu_principal():
    """Menú principal del sistema."""

    if os.path.exists(ARCHIVO_ESTADO):
        print("\n" + "=" * 40)
        print(f"ATENCIÓN: Se encontró un estado anterior guardado en '{ARCHIVO_ESTADO}'.")
        print("Esto podría contener datos de pruebas anteriores.")
        reset = input("¿Desea INICIAR DESDE CERO, eliminando el estado anterior? (S/N): ").strip().upper()
        if reset == 'S':
            try:
                os.remove(ARCHIVO_ESTADO)
                print("Estado anterior eliminado. Iniciando sistema vacío.")
            except Exception as e:
                print(f"Error al eliminar el archivo de estado: {e}")
        print("=" * 40)


    sistema = SistemaReservasPandas()

    while True:
        print("\n" + "=" * 40)
        print("      SISTEMA DE RESERVAS COWORKING (PANDAS)")
        print("=" * 40)
        print("[1] Registrar la reservación de una sala")
        print("[2] Editar el nombre del evento")
        print("[3] Consultar las reservaciones existentes y Exportar")
        print("[4] Registrar a un nuevo cliente")
        print("[5] Registrar una sala")
        print("[6] Generar Gráfico de Reservas Mensuales")
        print("[7] Salir (Guardar Estado)")
        print("-" * 40)

        opcion = input("Seleccione una opción: ").strip().upper()

        if opcion == '1':
            sistema.registrar_reservacion()
        elif opcion == '2':
            sistema.editar_reservacion()
        elif opcion == '3':
            sistema.consultar_reservaciones()
        elif opcion == '4':
            print("\n*** REGISTRAR NUEVO CLIENTE ***")

            # Validación SECUENCIAL para Cliente
            while True:
                nombre = input("Nombre del cliente: ")
                if not nombre.strip():
                    print("Error: El nombre del cliente no puede estar vacío o contener solo espacios. Intente de nuevo.")
                else:
                    break

            while True:
                apellidos = input("Apellidos del cliente: ")
                if not apellidos.strip():
                    print("Error: Los apellidos del cliente no pueden estar vacíos o contener solo espacios. Intente de nuevo.")
                else:
                    break

            sistema.registrar_cliente(nombre, apellidos)

        elif opcion == '5':
            print("\n*** REGISTRAR NUEVA SALA ***")

            # Validación SECUENCIAL para Sala
            while True:
                nombre = input("Nombre de la sala: ")
                if not nombre.strip():
                    print("Error: El nombre de la sala no puede estar vacío o contener solo espacios. Intente de nuevo.")
                else:
                    break

            while True:
                cupo = input("Cupo de la sala: ")
                if not cupo.strip():
                    print("Error: El cupo de la sala no puede estar vacío o contener solo espacios. Intente de nuevo.")
                elif not cupo.isdigit():
                    print("Error: El cupo debe ser un número entero. Intente de nuevo.")
                else:
                    break

            sistema.registrar_sala(nombre, cupo)

        elif opcion == '6':
            sistema.generar_grafica_reservas_mensuales()
        elif opcion == '7':
            confirmacion = input("¿Está seguro que desea salir? (S/N): ").upper()
            if confirmacion == 'S':
                print("Guardando estado...")
                sistema.guardar_estado()
                print("Saliendo del sistema. ¡Adiós!")
                break
            else:
                print("Volviendo al menú principal.")

        else:
            print("Opción no válida. Intente de nuevo.")


if __name__ == "__main__":
    menu_principal()


ATENCIÓN: Se encontró un estado anterior guardado en 'estado_reservas_pandas.json'.
Esto podría contener datos de pruebas anteriores.
¿Desea INICIAR DESDE CERO, eliminando el estado anterior? (S/N): s
Estado anterior eliminado. Iniciando sistema vacío.
No se encontró estado anterior. Se inicia con un estado inicial vacío.

      SISTEMA DE RESERVAS COWORKING (PANDAS)
[1] Registrar la reservación de una sala
[2] Editar el nombre del evento
[3] Consultar las reservaciones existentes y Exportar
[4] Registrar a un nuevo cliente
[5] Registrar una sala
[6] Generar Gráfico de Reservas Mensuales
[7] Salir (Guardar Estado)
----------------------------------------
Seleccione una opción: 7
¿Está seguro que desea salir? (S/N): s
Guardando estado...
Estado de la solución almacenado exitosamente.
Saliendo del sistema. ¡Adiós!
